In [1]:
import nbformat

notebook_path = "DeepEval.ipynb"

nb = nbformat.read(notebook_path, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

nbformat.write(nb, notebook_path)

print("Fixed!")


Fixed!


In [7]:
!pip install langchain-community langchain-text-splitters pypdf pymupdf langchain-huggingface sentence-transformers langchain-experimental portalocker posthog deepeval  --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 14.5 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
model_id = "Qwen/Qwen3-4B-Instruct-2507"

# ==========================================
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 確保 pipeline 正常運作的常見修復
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# ==========================================
# 建立原生 Transformers Pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,      # 決定回答長度
    temperature=0.2,         # RAG 建議設低一點，減少幻覺
    repetition_penalty=1.1,  # 避免鬼打牆
    return_full_text=False   # 重要：設為 False 只回傳生成的部分
)

# . 轉為 LangChain 的 LLM 物件
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# (關鍵) 轉為 ChatModel 物件
# ChatHuggingFace 會自動處理 prompt template (User/System/Assistant 格式)
chat_model = ChatHuggingFace(llm=llm)

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

In [5]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 43.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [9]:
from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams


In [10]:
from deepeval.models.base_model import DeepEvalBaseLLM

# 1. 定義包裝器 (Wrapper)
class LangChainJudge(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        # ChatModel (ChatHuggingFace) 接收字串或訊息列表
        # 並回傳 AIMessage 物件，我們需要取它的 .content 屬性
        try:
            res = self.model.invoke(prompt)
            return res.content
        except Exception as e:
            return f"Error: {e}"

    async def a_generate(self, prompt: str) -> str:
        # DeepEval 內部有時會呼叫非同步方法，這裡直接轉給同步方法即可
        return self.generate(prompt)

    def get_model_name(self):
        return "Local Qwen Model"

# 2. 實例化你的裁判
# 把前面準備好的 chat_model 傳進去
qwen_judge = LangChainJudge(model=chat_model)

##Test Correctness

In [12]:
#定義裁判 (The Judge)
correctness_metric=GEval(
    name="Correctness",
    model=qwen_judge,
    evaluation_params=[LLMTestCaseParams.EXPECTED_OUTPUT,LLMTestCaseParams.ACTUAL_OUTPUT],
    evaluation_steps=["Determine whether the actual output is factually correct based on the expected output."]
)

#準備考卷 (The Test Case)
great_answer="Madrid is the capital of Spain."
pred_answer="Madrid "
test_case_correctness=LLMTestCase(
    input="What is the capital of Spain?",
    expected_output=great_answer,
    actual_output=pred_answer
)

#開始評分 (Grading)

correctness_metric.measure(test_case_correctness)
print(f"得到的分數:{correctness_metric.score}")

Output()

Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


得到的分數:0.3


##Test faithfulness

In [16]:
question="what is 3+3?"
context=["16"]
generated_answer = "6"

faithfulness_metric=FaithfulnessMetric(
    threshold=0.7,
    model=qwen_judge,
    include_reason=False
)

test_case = LLMTestCase(
    input = question,
    actual_output=generated_answer,
    retrieval_context=context

)
faithfulness_metric.measure(test_case)
print(f"得到的分數:{faithfulness_metric.score}")
print(f"原因:{faithfulness_metric.reason}")

Output()

Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


得到的分數:0.0
原因:None


##Test contextual relevancy

In [18]:
actual_output = "then go somewhere else."
retrieval_context = ["this is a test context","mike is a cat","if the shoes don't fit, then go somewhere else."]
gt_answer = "if the shoes don't fit, then go somewhere else."

relevance_metric = ContextualRelevancyMetric(
    threshold=1,
    model=qwen_judge,
    include_reason=True
)
relevance_test_case = LLMTestCase(
    input="then go somewhere else.",
    actual_output=actual_output,
    retrieval_context=retrieval_context,
    expected_output=gt_answer
)
relevance_metric.measure(relevance_test_case)
print(f"得到的分數:{relevance_metric.score}")
print(f"原因:{relevance_metric.reason}")

Output()

Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


得到的分數:0.3333333333333333
原因:The score is 0.33 because while the statement 'if the shoes don't fit, then go somewhere else' is partially relevant, the majority of the context contains irrelevant content such as 'this is a test context' and'mike is a cat', which do not connect to the input, limiting overall relevance.


##Test two different cases together with several metrics together

In [22]:
import os

# 1. 增加超時時間 (原本預設可能只有幾分鐘，設為 1000 秒確保本地模型跑得完)
os.environ["DEEPEVAL_PER_TASK_TIMEOUT_SECONDS_OVERRIDE"] = "1000"

# 2. 如果你需要強制關閉並行處理 (讓它一個一個跑，減輕電腦負擔)
# 注意：這取決於 DeepEval 版本，若此環境變數無效，主要依賴上面的 Timeout 設定
os.environ["DEEPEVAL_ASYNC_MODE"] = "false"

In [24]:
new_test_case = LLMTestCase(
    input="What is the capital of Spain?",
    expected_output="Madrid is the capital of Spain.",
    actual_output="MadriD.",
    retrieval_context=["Madrid is the capital of Spain."]
)
#evaluate會自動執行前面定義過的
evaluate(
    test_cases=[relevance_test_case, new_test_case],
    metrics=[correctness_metric, faithfulness_metric, relevance_metric]
)

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using Local Qwen Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using Local Qwen Model, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using Local Qwen Model, strict=False, 
async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the 



Metrics Summary

  - ❌ Correctness [GEval] (score: 0.4, threshold: 0.5, strict: False, evaluation model: Local Qwen Model, reason: The actual output misses the conditional clause 'if the shoes don't fit' that is present in the expected output, making it factually incomplete. The core logical condition is omitted, so the response does not accurately reflect the intended meaning or structure of the expected output., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: Local Qwen Model, reason: None, error: None)
  - ❌ Contextual Relevancy (score: 0.3333333333333333, threshold: 1.0, strict: False, evaluation model: Local Qwen Model, reason: The score is 0.33 because while the statement 'if the shoes don't fit, then go somewhere else' is partially relevant, the majority of the context contains irrelevant content such as 'this is a test context' and'mike is a cat', which do not connect to the input, limiting overall relevance., error: None)

For te

⚠ WARNING: No hyperparameters logged.
» ]8;id=919404;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 275.64s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 2

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=False, score=0.4, reason="The actual output misses the conditional clause 'if the shoes don't fit' that is present in the expected output, making it factually incomplete. The core logical condition is omitted, so the response does not accurately reflect the intended meaning or structure of the expected output.", strict_mode=False, evaluation_model='Local Qwen Model', error=None, evaluation_cost=None, verbose_logs='Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Determine whether the actual output is factually correct based on the expected output."\n] \n \nRubric:\nNone \n \nScore: 0.4'), MetricData(name='Faithfulness', threshold=0.7, success=True, score=1.0, reason=None, strict_mode=False, evaluation_model='Local Qwen Model', error=None, evaluation_cost=None, verbose_logs='Truths (limit=None):\n[\n    "Mike is a cat.",\n    "If the sho

In [ ]:
import nbformat

notebook_path = "你的notebook名稱.ipynb"

nb = nbformat.read(notebook_path, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

nbformat.write(nb, notebook_path)

print("Fixed!")
